In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import ollama
import json
from summary_agent import SummaryAgent

In [3]:
def load_json(path):
    with open(path, 'r') as f:
        data = json.load(f)
    return data


i1_case =  load_json('../../ECHR/echr-processed/001-61521.json')
i2_case = load_json('../../ECHR/echr-processed/001-57416.json')
i3_case = load_json('../../ECHR/echr-processed/001-205675.json')
i4_case = load_json('../../ECHR/echr-processed/001-225326.json')

In [104]:
summary_agent = SummaryAgent()

In [105]:
i4_case.keys()

dict_keys(['itemid', 'importance', 'judgementdate', '__articles', '__conclusion', 'appno', 'article', 'extractedappno', 'docname', 'parties', 'rank', 'cite', 'procedure', 'facts', 'law', 'conclusion'])

In [109]:
summary_reposne = summary_agent.summarize_case(i1_case)

   init_summary ....


BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}

In [37]:
len(i1_case['facts'])

44700

In [100]:
from transformers import AutoTokenizer
from nltk import tokenize

def count_tokens(text):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokens = tokenizer.tokenize(text)
    num_tokens = len(tokens)
    return num_tokens

def get_sentences(text):
    sentences = tokenize.sent_tokenize(i1_case['facts'])
    merged_sentences = []

    sentence = ''
    for idx, i in enumerate(sentences):
        sentence += i
        if len(i) < 10:
            continue
        else:
            merged_sentences.append(sentence)
            sentence = ''
    
    return merged_sentences

def overlapping_split(text, token_limit=7500, overlap = 300):
    words = len(text.split(' '))
    num_tokens = count_tokens(text)
    sentences = get_sentences(text)
    print(f'text has {words} words and {len(sentences)} sentences, on average {words/len(sentences)} word/sentecesand {num_tokens} tokens, token limit is {token_limit}')
    
    if num_tokens <= token_limit:
        return text
    
    chuncks = (num_tokens // token_limit) + 1 + 1
    sentences_in_chunck = (len(sentences) // chuncks) - 10
    spliited_sentences = [sentences[i:i+sentences_in_chunck] for i in range(0, len(sentences), sentences_in_chunck)]
    return spliited_sentences


In [101]:
spliited_sentences = overlapping_split(i1_case['facts'])

Token indices sequence length is longer than the specified maximum sequence length for this model (9617 > 512). Running this sequence through the model will result in indexing errors


text has 7589 words and 335 sentences, on average 22.65373134328358 word/sentecesand 9617 tokens, token limit is 7500


In [102]:
len(spliited_sentences)

4

In [103]:
for i in spliited_sentences:
    sent = ' '.join(i)
    num_tokens = count_tokens(sent)
    print(len(i), num_tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (2023 > 512). Running this sequence through the model will result in indexing errors


101 2023


Token indices sequence length is longer than the specified maximum sequence length for this model (2714 > 512). Running this sequence through the model will result in indexing errors


101 2714


Token indices sequence length is longer than the specified maximum sequence length for this model (3310 > 512). Running this sequence through the model will result in indexing errors


101 3310


Token indices sequence length is longer than the specified maximum sequence length for this model (1567 > 512). Running this sequence through the model will result in indexing errors


32 1567


In [88]:
from nltk import tokenize

sentences = tokenize.sent_tokenize(i1_case['facts'])
merged_sentences = []

sentence = ''
for idx, i in enumerate(sentences):
    sentence += i
    if len(i) < 10:
        continue
    else:
        merged_sentences.append(sentence)
        sentence = ''

In [89]:
len(merged_sentences)

335

In [90]:
for i in merged_sentences:
    print(len(i))

34
59
120
134
56
26
155
35
78
66
48
101
98
47
63
104
142
63
67
62
29
129
39
78
185
66
69
114
120
59
79
86
82
37
17
100
183
72
98
78
220
47
110
29
115
217
117
58
239
68
64
111
43
103
56
189
97
25
45
44
150
41
82
73
40
104
15
175
91
117
45
61
42
23
151
59
36
78
118
188
37
82
100
34
105
13
158
30
14
95
26
163
23
104
18
142
217
45
129
46
64
107
168
41
111
70
129
113
94
155
104
120
92
260
56
104
284
72
209
30
78
94
71
45
56
58
92
64
48
23
30
98
68
90
26
41
87
103
142
24
75
81
185
105
85
195
197
70
50
113
34
238
89
120
144
71
148
81
93
146
345
168
175
248
118
91
257
177
140
144
94
104
111
123
80
93
215
294
335
192
92
264
158
110
150
264
74
343
23
300
260
107
52
225
271
154
209
127
109
224
75
155
41
135
45
171
142
31
25
270
183
18
167
123
30
37
27
42
196
158
242
269
29
24
88
30
25
91
270
265
185
315
18
253
29
281
57
107
76
231
42
220
128
192
135
161
186
148
35
62
69
185
87
187
62
264
179
52
288
285
354
291
293
338
179
168
154
56
207
225
318
47
153
162
27
142
25
109
39
65
106
36
13
216
26
66
6

In [41]:
overlapping_split(i1_case['facts'])

Token indices sequence length is longer than the specified maximum sequence length for this model (9617 > 512). Running this sequence through the model will result in indexing errors


text has 44700 words and 9617 tokens, token limit is 7500


In [42]:
sentences = i1_case['facts'].split('.')
len(sentences)

670

In [56]:
sentences[1]

'THE CIRCUMSTANCES OF THE CASE\n9.'

In [74]:
len(sentences), len(merged_sentences)

(450, 335)

In [77]:
merged_sentences[0]

'I.THE CIRCUMSTANCES OF THE CASE\n9.'

In [16]:
with open('./i4_case.json', 'w') as f:
    json.dump(summary_reposne, f, indent=4)
